#### ID 2061

```Count the number of users who made more than 5 searches in August 2021.```

In [ ]:
%%sql
SELECT COUNT(user_id)
FROM (SELECT user_id, COUNT(search_id) AS cnt
      FROM fb_searches
      WHERE EXTRACT(MONTH FROM date) = 8
        AND EXTRACT(YEAR FROM date) = 2021
      GROUP BY user_id
      HAVING COUNT(search_id) > 5) AS sq

In [ ]:
df = fb_searches
df.query('date.dt.year == 2021 & date.dt.month == 8').groupby('user_id', as_index=False).agg(cnt=('search_id', 'count')).query('cnt > 5')['user_id'].count()

#### ID 2062

```How many searches were there in the second quarter of 2021?```

In [ ]:
%%sql
SELECT COUNT(*) AS result
FROM fb_searches
WHERE EXTRACT(QUARTER FROM date) = 2
  AND EXTRACT(YEAR FROM date) = 2021

In [ ]:
df = fb_searches
df.query('date.dt.year == 2021 & date.dt.quarter == 2')['search_id'].nunique()

#### ID 2063

```You are given a list of exchange rates from various currencies to US Dollars (USD) in different months. Show how the exchange rate of all the currencies changed in the first half of 2020. Output the currency code and the difference between values of the exchange rate between July 1, 2020 and January 1, 2020.```

In [ ]:
%%sql
SELECT source_currency,
       AVG(exchange_rate) FILTER ( WHERE date = '2020-07-01') -
       AVG(exchange_rate) FILTER ( WHERE date = '2020-01-01') AS difference
FROM sf_exchange_rate
GROUP BY source_currency

In [ ]:
df = sf_exchange_rate
jan_rate_df = df.query('date == "2020-01-01"')
jul_rate_df = df.query('date == "2020-07-01"')

df = pd.merge(jul_rate_df, jan_rate_df, how='inner', on='source_currency', suffixes=['_jul', '_jan'])

df['difference'] = df['exchange_rate_jul'] - df['exchange_rate_jan']

df[['source_currency', 'difference']]

#### ID 2067

```What percentage of all products are both low fat and recyclable?```

In [ ]:
%%sql
SELECT COUNT(product_id) FILTER (WHERE is_low_fat = 'Y' AND is_recyclable = 'Y') * 100.0 /
       COUNT(product_id) AS percentage
FROM facebook_products;

In [ ]:
df = facebook_products
cnt_products = df['product_id'].count()

filtered_cnt_products = df.query('is_low_fat == "Y" & is_recyclable == "Y"')['product_id'].count()

result = filtered_cnt_products * 100 / cnt_products